In [2]:
# mudando o diretório
%cd /home/karem/Artigo_Periodico/Detector/
# apaga os arquivos da pasta label
!rm -rf Ensemble/labels

# cria o direório
!mkdir -p Ensemble/labels/YoloX/
# copia os rótulos para a pasta Ensemble
!cp -r YoloX/Data_trained/YOLOX_outputs_11/yolox_voc_s/label/. Ensemble/labels/YoloX/

!mkdir -p Ensemble/labels/YoloV5/
!cp -r YoloV5/Data_trained/runs_21/detect/exp/labels/. Ensemble/labels/YoloV5/

# retorna para o diretório atual 
%cd Ensemble

/home/karem/Artigo_Periodico/Detector
/home/karem/Artigo_Periodico/Detector/Ensemble


In [3]:
import pandas as pd
import os

import numpy as np
from ensemble_boxes import *
import glob
from glob import glob
from tqdm import tqdm

In [4]:
# Obtém o diretório atual

path = os.getcwd() + '/'
print(path)

/home/karem/Artigo_Periodico/Detector/Ensemble/


In [5]:
# Cria o dietório caso não exista
def create_folders(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
    
# se houver a pasta, ela é removida
import shutil
def remove_folders(path):
    if os.path.exists(path):
        shutil.rmtree(path)

In [6]:
# Convert lista para string
def convert2(lst):    
    return ' '.join(lst)

# função que volta o tamanho do bounding box para orginal
# o tamanho salvo foi normalizado entre 0 e 1
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]

    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int

    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height

    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]

    return bboxes

In [7]:
# Carrega o arquivo com o nome e tamanho das imagens do conjunto de teste

# df = pd.read_csv('/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/Folder_1_Covid/test.csv')
df = pd.read_csv('/home/karem/Artigo_Periodico/1_Dataset/CSV_Files/2class_trainCovidNormal/test.csv')

df_test = df.copy()
df_test['id'] = df_test.id.replace('_image', '.txt', regex=True) # troca o rótulo pela extensão

print(len(df_test))
df_test.head(5)

564


,id,class_name,width,height,xmin,ymin,xmax,ymax,width_roi,height_roi
0,888109901b29.txt,indeterminate,640,640,"[88.0, 332.0]","[158.0, 179.0]","[276.0, 499.0]","[413.0, 506.0]","[188.0, 167.0]","[255.0, 327.0]"
1,35a789322c1d.txt,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
2,462a64ef01e3.txt,negative,640,640,[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]
3,952d4a77beda.txt,indeterminate,640,640,[404.0],[335.0],[517.0],[459.0],[113.0],[124.0]
4,96417c620956.txt,typical,640,640,"[101.0, 401.0]","[214.0, 258.0]","[212.0, 513.0]","[473.0, 547.0]","[111.0, 111.0]","[259.0, 289.0]"


In [8]:
image_ids = []
PredictionStrings = []
path_labels_yolo = path + 'labels/YoloV5/*.txt'


path_labels_pred = path + 'labels/YoloV5_2/'
remove_folders(path_labels_pred)
create_folders(path_labels_pred)

for file_path in tqdm(glob(path_labels_yolo)):    
    image_id = file_path.split('/')[-1].split('.')[0] + '_image'
    
    w, h = df_test[['width', 'height']].values[0]
    
    for i in range(len(df_test)):
        if image_id == df_test['id'][i]:
            w = int(df_test['width'][i])
            h = int(df_test['height'][i])
            break
    
    f = open(file_path, 'r')    
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
#     print(data)
    
    # Uma casa decimal para o confidence score
#     bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    
    # Duas casas
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 2).astype(str))
#     print(bboxes)   
    
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
     
    # cria um novo txt com os tamanhos originais dos BB
    nome_file = os.path.basename(file_path)
    file = open(path_labels_pred + nome_file, "w")     
    for i in range(int(len(bboxes)/6)):
        result = convert2(bboxes[i*6:i*6+6])
        file.write(result + '\n')
        #print(result)
#         print(bboxes[i*6:i*6+6])
    file.close() 
       
    
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

    pred_df = pd.DataFrame({'id':image_ids,
                        'PredictionString':PredictionStrings})
pred_df


100%|██████████| 349/349 [00:00<00:00, 483.57it/s]


,id,PredictionString
0,8f3260527b5d_image,0 0.43 117 294 282 433 0 0.54 385 90 498 422
1,7f6ad0122fc2_image,0 0.4 444 213 564 402
2,de8c50c2ae86_image,0 0.29 128 155 250 322 0 0.32 118 169 239 465 ...
3,ba77c4831e02_image,0 0.33 79 366 252 528 0 0.63 363 326 538 540
4,39acf134b0a1_image,0 0.28 111 127 256 427 0 0.42 388 234 532 435
...,...,...
344,cc7023fca3c2_image,0 0.28 426 113 612 509 0 0.31 129 59 340 499
345,b98174b36c85_image,0 0.39 31 106 179 442
346,ce435a634ee1_image,0 0.4 65 142 179 388
347,b1c963b7c90a_image,0 0.32 64 174 170 409 0 0.39 344 333 459 488


In [9]:
#### Converte a linha (que é uma lista) de string em um vetor do tipo float

import re
def convert_df_lista_int(str1):    
#     str1 = re.sub(r"[\([{})\]]", "", str1) # remove os caracteres da string
    #str1 = (str1.replace('.0', '')) # substitui a casa decimal .0 por vazio, tornando a string no formato int
    str1 = list(str1.split(" "))    
    str1 = [(float(i)) for i in str1] # convert os valores para inteiro e depois string    
    return str1

In [10]:
# variáveis para o ensemble
iou_thr = 0.4
skip_box_thr = 0.0001
sigma = 0.1
weights = [2, 1]

size_image = 640

In [14]:
def data_detector(i, diretory, df):
    matrix = []
    label = []
    score = []
    bbox = []
    
    # verifica se o arquivo txt existe, ou seja, se o detector achou alguma coisa na imagem
    if (os.path.isfile(diretory + df['id'][i])): 
        with open(os.path.join(diretory + df['id'][i]), 'r') as f: # abre o txt
            
            for line in f:
                if line: #avoid blank lines                    
                    linha = convert_df_lista_int(line)                    
                matrix.append(linha) # obtém a informação da linha toda (label, conf score, bboxes)
                
                #### Separa as listas (classe, conf. score e bbox) para cada detector
                label.append(linha[0]) # obtém o rótulo da classe
                score.append(linha[1]) # obtém o conf. score
                bbox.append(linha[2:6]) # obtém o bbox
                                    
                               
    else: # se não existir, cria uma linha de informações vazias
        linha = [1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
        label.append(linha[0]) # obtém o rótulo da classe
        score.append((linha[1] * 2/3)) # obtém o conf. score
        bbox.append(linha[2:6]) # obtém o bbox
        matrix = linha
        
#     print(diretory + df['id'][i])
#     print(matrix)    
        
        
    return matrix, linha, label, score, bbox
    

In [15]:
# Convert lista para string
def convert(lst):
    i = ''.join(lst)
    str1 = re.sub(r"[\([{})\]]", "", i)
    return str1

In [18]:
### Diretórios dos rótulos

dir_yolov5 = path + 'labels/YoloV5_2/'
dir_yolox = path + 'labels/YoloX/'

remove_folders(path + 'labels/2detectors')
create_folders(path + 'labels/2detectors')
    

for i in range(len(df_test)):
            
    
    matrix1, linha1, label1, score1, bbox1 = data_detector(i, dir_yolov5, df_test)
    matrix2, linha2, label2, score2, bbox2 = data_detector(i, dir_yolox, df_test)
    # print(matrix2)
    
    
    # print(matrix1, '  ----  ', matrix2, '\n\n')
            
    bbox1 = (np.array(bbox1) / size_image) # --> normaliza os dados para a entrada da função
    bbox2 = (np.array(bbox2) / size_image)
    # print(bbox1, '  ----  ', bbox2, '\n\n')
    
    ###### aplica o weighted boxes fusion
    boxes, scores, labels = weighted_boxes_fusion([bbox1, bbox2], [score1, score2], 
                                              [label1, label2], weights=weights, iou_thr=iou_thr, 
                                              skip_box_thr=skip_box_thr)
    
    boxes = (np.array(boxes) * size_image).tolist() # volta o bbox ao tamanho original   
    # print(boxes, '\n\n')
    
    
    ######### Outras opções
    # boxes, scores, labels = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
    # boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    

    ### Caso o detector não encontre nada na imagem, é colocado o bbox do outro detector 
    # para que a região demarcada não seja removida
    if (linha1[2] == 0):
        boxes = bbox2
        scores = score2
        labels = label2
    if (linha2[2] == 0):
        boxes = bbox1
        scores = score1
        labels = label1

    
    # print(boxes)
    # print(scores)
    # print(labels)
    # print('\n\n')
    
    #### organiza os dados pós ponderação dos detectores
    ## [label, conf score, x1, y1, x2, y2] 
    matrix = []    
    for j in range(len(labels)):
        linha = []
        linha.append(labels[j])
        linha.append(scores[j])
        linha.append(int(boxes[j][0])) # arredonda os valores após a ponderação
        linha.append(int(boxes[j][1]))
        linha.append(int(boxes[j][2]))
        linha.append(int(boxes[j][3]))
        matrix.append(linha)
    # print(matrix)
        
    
    ### Prepara para salvar os dados
    # cria um novo txt com os tamanhos originais dos BB
    nome_file = df_test['id'][i]
    file = open(path + 'labels/2detectors/' + nome_file, "w")     
    for j in range(len(matrix)):     
                
        result = (convert(str(matrix[j])).replace(',', '')) # transforma a lista em string
        file.write(result + '\n')
       
    file.close() 
    
    
    # Remove as linhas com bbox nulo ["0 0 0 0"]
    with open(path + 'labels/2detectors/' + nome_file, "r+") as f:
        new_f = f.readlines()
        f.seek(0)
        for line in new_f:
            if "0 0 0 0" not in line:
                f.write(line)
        f.truncate()
    f.close()
    
    # Caso o arquivo esteja vazio, insere uma linha com zeros
    with open(path + 'labels/2detectors/' + nome_file, "r+") as f:
        if os.path.getsize(path + 'labels/2detectors/' + nome_file) == 0:
            result = (convert(str([1, 1, 0, 0, 0, 0])).replace(',', '')) # transforma a lista em string
            f.write(result + '\n')
    f.close()
        
    
    
# print(matrix1, '  ----  ', matrix2)
    

/home/karem/anaconda3/envs/env_py39/lib/python3.9/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0. 0. 0. 0.].
  warnings.warn("Zero area box skipped: {}.".format(box_part))


In [ ]:
### Diretórios dos rótulos

dir_yolov5 = path + 'labels/YoloV5/'
dir_yolox = path + 'labels/YoloX/'


for i in range(len(df_test)):
    
    matrix1 = []
    label1 = []
    score1 = []
    bbox1 = []
    matrix2 = []
    label2 = []
    score2 = []
    bbox2 = []
            
    
    # verifica se o arquivo txt existe, ou seja, se o detector achou alguma coisa na imagem
    if (os.path.isfile(dir_yolov5 + df_test['id'][i])): 
        with open(os.path.join(dir_yolov5 + df_test['id'][i]), 'r') as f1: # abre o txt
            
            for line in f1:
                if line: #avoid blank lines
                    linha1 = convert_df_lista_int(line)
                matrix1.append(linha1) # obtém a informação da linha toda (label, conf score, bboxes)
                
                #### Separar listas (classe, conf. score e bbox) para cada detector
                label1.append(linha1[0]) # obtém o rótulo da classe
                score1.append(linha1[1]) # obtém o conf. score
                bbox1.append(linha1[2:6]) # obtém o bbox
                
#                 print(bbox1)
                               
    else: # se não existir, cria uma linha de informações vazias
        linha1 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
        label1.append(linha1[0]) # obtém o rótulo da classe
        score1.append((linha1[1] * 2/3)) # obtém o conf. score
        bbox1.append(linha1[2:6]) # obtém o bbox
    
    
    
    ### Segundo detector
    # verifica se o arquivo txt existe, ou seja, se o detector achou alguma coisa na imagem
    if (os.path.isfile(dir_yolox + df_test['id'][i])): 
        with open(os.path.join(dir_yolox + df_test['id'][i]), 'r') as f2: # abre o txt
            
            for line in f2:
                if line: #avoid blank lines
                    linha2 = convert_df_lista_int(line)
                matrix2.append(linha2) # obtém a informação da linha toda (label, conf score, bboxes)
                
                #### Separar listas (classe, conf. score e bbox) para cada detector
                label2.append(linha2[0]) # obtém o rótulo da classe
                score2.append((linha2[1] * 2/3)) # obtém o conf. score
                bbox2.append(linha2[2:6]) # obtém o bbox
                               
    else: # se não existir, cria uma linha de informações vazias
        linha2 = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]        
        label2.append(linha2[0]) # obtém o rótulo da classe
        score2.append(linha2[1]) # obtém o conf. score
        bbox2.append(linha2[2:6]) # obtém o bbox
    
    
    print(matrix1, '  ----  ', matrix2)
            
        
    
    ###### aplica o weight boxes fusion
    boxes, scores, labels = weighted_boxes_fusion([bbox1, bbox2], [score1, score2], 
                                              [label1, label2], weights=weights, iou_thr=iou_thr, 
                                              skip_box_thr=skip_box_thr)

    ### Caso o detector não encontre nada na imagem, é colocado o bbox do outro detector 
    # para que a região demarcada não seja removida
    if (linha1[2] == 0):
        boxes = bbox2
        scores = score2
        labels = label2
    if (linha2[2] == 0):
        boxes = bbox1
        scores = score1
        labels = label1

    
    print(boxes)
    print(scores)
    print(labels)
    print('\n\n')
    
    
    
    
# print(bbox1, '  ----  ', bbox2)
#     print(matrix1, '  ----  ', matrix2)
    
    
   